#ETL - Loterias

O objetivo desse exercício foi extrair dados de um site, transformá-los e retornar um resumo para o usuário.

##Import

In [1]:
!pip install requests

In [2]:
import requests
import pandas as pd
import collections

#Extract

In [5]:
#Coleta os dados brutos
url = 'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados?modalidade=Lotof%C3%A1cil'
r = requests.get(url, verify=False)
r_text = r.text
r_text[:300]

/home/lucas/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'servicebus2.caixa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'{\r\n  "html": "<table class=\\"tabela-resultado lotofacil\\">\\r\\n<thead>\\r\\n<tr>\\r\\n<th><small>\\r\\nConcurso\\r\\n</small></th><th><small>\\r\\nData_Sorteio\\r\\n</small></th><th><small>\\r\\nBola1\\r\\n</small></th><th><small>\\r\\nBola2\\r\\n</small></th><th><small>\\r\\nBola3\\r\\n</small></th><th><small>\\r\\nBola4\\r\\n'

In [ ]:
#Cria um Dataframe
df = pd.read_html(r_text) #lê os dados
df = df[0].copy() #cria o dataframe
df = df.dropna()  #remove os nulos
df.columns = [col[4:-4] for col in df.columns]  #renomeia as colunas
df.head()

,Concurso,Data_Sorteio,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,...,Ganhadores_12_Números,Ganhadores_11_Números,Valor_Rateio_15_Números,Valor_Rateio_14_Números,Valor_Rateio_13_Números,Valor_Rateio_12_Números,Valor_Rateio_11_Números,Acumulado_15_Números,Estimativa_Prêmio,Valor_Acumulado_Especial
0,1,29/09/2003,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,...,48807.0,257593.0,"R$49.765,82","R$689,84","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
4,2,06/10/2003,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,...,81252.0,478188.0,"R$596.323,70","R$1.388,95","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
6,3,13/10/2003,1.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,...,96244.0,608211.0,"R$400.623,70","R$2.173,36","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
8,4,20/10/2003,1.0,2.0,4.0,5.0,8.0,10.0,12.0,13.0,...,123912.0,706657.0,"R$902.226,02","R$1.498,72","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
10,5,27/10/2003,1.0,2.0,4.0,8.0,9.0,11.0,12.0,13.0,...,195636.0,860992.0,"R$380.017,55","R$687,49","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"


#Transform

In [ ]:
#Identifica se um número é par ou ímpar
def qual_tipo(num):
  return (num % 2) == 0

In [ ]:
#Transformações - Parte 1
numeros = []
cont_pares = []
cont_impares = []
row_pares = []

for df_idx, row in df.iloc[:50,:18].iterrows(): #seleciona algumas linhas e colunas
  for row_idx in row.index: #itera nos indices das linhas
    if row_idx.startswith('Bola'):  #indice inicia com 'Bola'
      num = row.loc[row_idx]  #coleta o numero
      tipo = qual_tipo(num) #testa se é par(=True)
      numeros.append(num)
      row_pares.append(tipo)  #add o resultado na lista temporária da linha
  
  total_p_row = sum(row_pares) #soma os pares da linha
  cont_pares.append(total_p_row)  #add o valor a lista geral
  cont_impares.append(15-total_p_row)  #add de impares a lista geral
  row_pares.clear() #limpa a lista de pares
  total_p_row = 0 #zera a soma


In [ ]:
#Transformações - Parte 2
counter = collections.Counter(numeros)
resultado = pd.DataFrame(counter.items(), columns=['Número', 'Frequencia'])
resultado['p_freq'] = resultado['Frequencia']/resultado['Frequencia'].sum()
resultado = resultado.sort_values(by='p_freq')
resultado.head()

,Número,Frequencia,p_freq
18,12.0,22,0.029333
10,18.0,22,0.029333
1,3.0,23,0.030667
24,22.0,24,0.032000
19,15.0,25,0.033333


In [ ]:
print('''
O número mais frequente é o: {}
O número menos frequente é o: {}
O máximo e mínimo de pares em uma rodada é, respectivamente: {} e {}
O máximo e mínimo de impares em uma rodada é, respectivamente: {} e {}

'''.format(resultado.iloc[-1,0], resultado.iloc[0,0], max(cont_pares), min(cont_pares), max(cont_impares), min(cont_impares))
)



O número mais frequente é o: 19.0
O número menos frequente é o: 12.0
O máximo e mínimo de pares em uma rodada é, respectivamente: 9 e 5
O máximo e mínimo de impares em uma rodada é, respectivamente: 10 e 6


